In [1]:
#loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setting plt to inline
%matplotlib inline

In [2]:
# importing sqlite3
import sqlite3

# conneting to db and creating cursor
conn = sqlite3.Connection('data/movies.db')
cur = conn.cursor()

# creating query functions
def fetcha(q):
    return cur.execute(q).fetchall()
def fetcho(q):
    return cur.execute(q).fetchone()

In [3]:
# getting table names
q = """SELECT name FROM sqlite_master 
WHERE type IN ('table','view') 
AND name NOT LIKE 'sqlite_%'
ORDER BY 1"""
fetcha(q)

[('bom_movie_gross',),
 ('imdb_name_basic',),
 ('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',),
 ('rotten_tomatoes_critic_reviews',),
 ('rotten_tomatoes_movies',),
 ('tmdb_movies',),
 ('tn_movie_budgets',)]

In [7]:
q = "PRAGMA table_info('imdb_title_principals')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'tconst', 'TEXT', 0, None, 0),
 (2, 'ordering', 'INTEGER', 0, None, 0),
 (3, 'nconst', 'TEXT', 0, None, 0),
 (4, 'category', 'TEXT', 0, None, 0),
 (5, 'job', 'TEXT', 0, None, 0),
 (6, 'characters', 'TEXT', 0, None, 0)]

In [11]:
q = "PRAGMA table_info('imdb_name_basics')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 0),
 (1, 'nconst', 'TEXT', 0, None, 0),
 (2, 'primary_name', 'TEXT', 0, None, 0),
 (3, 'birth_year', 'REAL', 0, None, 0),
 (4, 'death_year', 'REAL', 0, None, 0),
 (5, 'primary_profession', 'TEXT', 0, None, 0),
 (6, 'known_for_titles', 'TEXT', 0, None, 0)]

In [9]:
q = "PRAGMA table_info('imdb_title_basics')"
fetcha(q)

[(0, 'idx', 'INTEGER', 0, None, 1),
 (1, 'tconst', 'TEXT', 0, None, 0),
 (2, 'primary_title', 'TEXT', 0, None, 0),
 (3, 'original_title', 'TEXT', 0, None, 0),
 (4, 'start_year', 'INTEGER', 0, None, 0),
 (5, 'runtime_minutes', 'REAL', 0, None, 0),
 (6, 'genres', 'TEXT', 0, None, 0)]

In [24]:
# creating initial dataframe by joining the three tables and printing first row
q = """SELECT*FROM imdb_name_basics
       JOIN imdb_title_principals 
       USING(nconst)
       JOIN imdb_title_basics AS ib
       USING(tconst)
       JOIN bom_movie_gross AS bm
       ON ib.primary_title = bm.title
       """
df = pd.DataFrame(fetcha(q))
df.columns = [i[0] for i in cur.description]
df.head(20)     

,idx,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,idx,tconst,ordering,...,original_title,start_year,runtime_minutes,genres,idx,title,studio,domestic_gross,foreign_gross,year
0,0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",466550,tt2398241,9,...,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",2804,Smurfs: The Lost Village,Sony,45000000.0,152200000,2017
1,8,nm0063618,Jeff Beal,1963.0,NaN,"composer,music_department,soundtrack","tt0183659,tt2545118,tt0384766,tt1856010",59770,tt2125666,7,...,The Queen of Versailles,2012,100.0,Documentary,956,The Queen of Versailles,Magn.,2400000.0,None,2012
2,8,nm0063618,Jeff Beal,1963.0,NaN,"composer,music_department,soundtrack","tt0183659,tt2545118,tt0384766,tt1856010",725124,tt5278596,8,...,Weiner,2016,96.0,Documentary,2555,Weiner,IFC,1700000.0,None,2016
3,8,nm0063618,Jeff Beal,1963.0,NaN,"composer,music_department,soundtrack","tt0183659,tt2545118,tt0384766,tt1856010",722214,tt6322922,10,...,An Inconvenient Sequel: Truth to Power,2017,98.0,Documentary,2934,An Inconvenient Sequel: Truth to Power,Par.,3500000.0,1900000,2017
4,31,nm0070822,Terry Benedict,NaN,NaN,"producer,director,assistant_director","tt0088247,tt2119532,tt0117280,tt0302427",101253,tt2119532,8,...,Hacksaw Ridge,2016,139.0,"Biography,Drama,History",2370,Hacksaw Ridge,LGF,67200000.0,108100000,2016
5,39,nm0074009,Richard Berge,NaN,NaN,"producer,miscellaneous,writer","tt0997088,tt1990352,tt5278460,tt6322922",217184,tt1990352,3,...,The Island President,2011,101.0,Documentary,1077,The Island President,Gold.,76400.0,None,2012
6,39,nm0074009,Richard Berge,NaN,NaN,"producer,miscellaneous,writer","tt0997088,tt1990352,tt5278460,tt6322922",722221,tt6322922,7,...,An Inconvenient Sequel: Truth to Power,2017,98.0,Documentary,2934,An Inconvenient Sequel: Truth to Power,Par.,3500000.0,1900000,2017
7,63,nm0083851,Sam Bisbee,NaN,NaN,"producer,composer,soundtrack","tt3813310,tt5655222,tt1990314,tt4158876",124068,tt1969062,6,...,Infinitely Polar Bear,2014,90.0,"Comedy,Drama,Romance",2096,Infinitely Polar Bear,SPC,1400000.0,None,2015
8,63,nm0083851,Sam Bisbee,NaN,NaN,"producer,composer,soundtrack","tt3813310,tt5655222,tt1990314,tt4158876",40963,tt1990314,9,...,Robot & Frank,2012,89.0,"Comedy,Crime,Drama",943,Robot & Frank,Gold.,3300000.0,None,2012
9,63,nm0083851,Sam Bisbee,NaN,NaN,"producer,composer,soundtrack","tt3813310,tt5655222,tt1990314,tt4158876",491129,tt2920808,10,...,God's Pocket,2014,88.0,"Crime,Drama",1799,God's Pocket,IFC,170000.0,None,2014


In [25]:
df.isna().sum()

idx                       0
nconst                    0
primary_name              0
birth_year            14676
death_year            30512
primary_profession      397
known_for_titles        193
idx                       0
tconst                    0
ordering                  0
category                  0
job                   20752
characters            18651
idx                       0
primary_title             0
original_title            0
start_year                0
runtime_minutes        1021
genres                  145
idx                       0
title                     0
studio                   30
domestic_gross          227
foreign_gross         12232
year                      0
dtype: int64

In [26]:
df['primary_profession'] = df['primary_profession'].astype('str')

In [27]:
actors_df = df[df['primary_profession'].str.contains('actor')]
actors_df.head()

,idx,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,idx,tconst,ordering,...,original_title,start_year,runtime_minutes,genres,idx,title,studio,domestic_gross,foreign_gross,year
10,91,nm0093589,Matt Bomer,1977.0,NaN,"actor,producer,soundtrack","tt1637688,tt2268016,tt1915581,tt1684226",51094,tt2268016,3,...,Magic Mike XXL,2015,115.0,"Comedy,Drama,Music",1927,Magic Mike XXL,WB,66000000.0,56500000,2015
11,91,nm0093589,Matt Bomer,1977.0,NaN,"actor,producer,soundtrack","tt1637688,tt2268016,tt1915581,tt1684226",427530,tt3799694,4,...,The Nice Guys,2016,116.0,"Action,Comedy,Crime",2423,The Nice Guys,WB,36300000.0,26500000,2016
16,116,nm0101248,Peter Bowles,1936.0,NaN,"actor,writer,producer","tt0060176,tt0144053,tt0078703,tt0200465",350089,tt2560102,4,...,Lilting,2014,83.0,"Drama,Romance",1840,Lilting,Strand,27100.0,None,2014
18,157,nm0116263,Francesco Bruni,1961.0,NaN,"writer,director,actor","tt6189344,tt2465578,tt0122648,tt1821597",438710,tt2465578,7,...,Il capitale umano,2013,111.0,Drama,2187,Human Capital,FM,159000.0,None,2015
19,157,nm0116263,Francesco Bruni,1961.0,NaN,"writer,director,actor","tt6189344,tt2465578,tt0122648,tt1821597",659290,tt4932388,6,...,Lasciati andare,2017,99.0,Comedy,3377,Let Yourself Go,Men.,12200.0,None,2018


In [28]:
directors_df = df[df['primary_profession'].str.contains('director')]
directors_df.head()

,idx,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,idx,tconst,ordering,...,original_title,start_year,runtime_minutes,genres,idx,title,studio,domestic_gross,foreign_gross,year
4,31,nm0070822,Terry Benedict,NaN,NaN,"producer,director,assistant_director","tt0088247,tt2119532,tt0117280,tt0302427",101253,tt2119532,8,...,Hacksaw Ridge,2016,139.0,"Biography,Drama,History",2370,Hacksaw Ridge,LGF,67200000.0,108100000,2016
12,107,nm0099841,Gilles Bourdos,1963.0,NaN,"writer,director","tt5539466,tt2150332,tt0940580,tt0339204",110691,tt2150332,5,...,Renoir,2012,111.0,"Biography,Drama,History",1334,Renoir,Gold.,2300000.0,None,2013
13,114,nm0101047,David Bowers,NaN,NaN,"animation_department,director,writer","tt2023453,tt1650043,tt0375568,tt0424095",178123,tt1650043,5,...,Diary of a Wimpy Kid: Rodrick Rules,2011,99.0,"Comedy,Family",417,Diary of a Wimpy Kid: Rodrick Rules,Fox,52700000.0,19700000,2011
14,114,nm0101047,David Bowers,NaN,NaN,"animation_department,director,writer","tt2023453,tt1650043,tt0375568,tt0424095",26879,tt2023453,5,...,Diary of a Wimpy Kid: Dog Days,2012,94.0,"Comedy,Family",810,Diary of a Wimpy Kid: Dog Days,Fox,49000000.0,28100000,2012
15,114,nm0101047,David Bowers,NaN,NaN,"animation_department,director,writer","tt2023453,tt1650043,tt0375568,tt0424095",601380,tt6003368,5,...,Diary of a Wimpy Kid: The Long Haul,2017,91.0,"Comedy,Family",2873,Diary of a Wimpy Kid: The Long Haul,Fox,20700000.0,19400000,2017
